<a href="https://colab.research.google.com/github/SoponUdomphon/my-learning-space/blob/main/cs-fundamental/01_memory_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛠️ **Hands-on Lab: Server Rescue Mission**
**สถานการณ์**: คุณพึ่งเข้าทำงานที่บริษัทแห่งหนึ่ง และได้รับมอบหมายให้แก้บั๊กในระบบจัดการนิยาย (Novel Management System) ที่มีปัญหาดังนี้:

1.  **Data Consistency:** แก้ข้อมูลนิยายเรื่องหนึ่ง แต่อีกเรื่องดันเปลี่ยนตาม?
2.  ** Performance:** ระบบโหลดเนื้อหานิยายช้ามากจน User บ่น
3.  ** Memory Leak:** Server แรมเต็มต้อง Restart ทุก 3 วัน


### **🛑 Station 1: The "Clone" Trap (Mutable & Reference)**
**โจทย์:** เรามี Cache เก็บรายชื่อนิยายแนะนำ แต่พอ Admin แก้ Tag ของ List A ทำไม List B (ที่เป็น Backup) ถึงเปลี่ยนตาม?

In [1]:
# 1. จำลองข้อมูล (Mutable Object)
top_novels = ["Harry Potter", "Percy Jackson"]
backup_novels = top_novels # <-- จุดเริ่มต้นของปัญหา

print(f"Original IDs: {id(top_novels)} vs {id(backup_novels)}")
print(f"Top & Backup IDs are same?: {top_novels is backup_novels}")

Original IDs: 136495098644160 vs 136495098644160
Top & Backup IDs are same?: True


In [4]:
# 2. เหตุการณ์: Admin เพิ่มนิยายเข้า List หลัก
top_novels.append("Lord of the Rings")

In [5]:
# 3. ตรวจสอบผลลัพธ์
print(f"Top: {top_novels}")
print(f"Backup: {backup_novels}") # ผลลัพธ์: Backup เปลี่ยนตามเฉยเลย!

Top: ['Harry Potter', 'Percy Jackson', 'Lord of the Rings', 'Lord of the Rings']
Backup: ['Harry Potter', 'Percy Jackson', 'Lord of the Rings', 'Lord of the Rings']


In [6]:
# ✅ Mission 1 Fix: จงแก้ไขบรรทัด 'backup_novels = ...'
# ให้เป็นการ Copy จริงๆ (ไม่ใช่แค่ส่ง Reference)
# Hint: ใช้ Slicing [:] หรือ .copy()

✅ Station 1: Solution

In [7]:
# 1. จำลองข้อมูล (Mutable Object)
top_novels = ["Harry Potter", "Percy Jackson"]
backup_novels = top_novels.copy() # <-- แก้ไขด้วยการใช้ .copy()

print(f"Original IDs: {id(top_novels)} vs {id(backup_novels)}")
print(f"Top & Backup IDs are same?: {top_novels is backup_novels}") # <- ตรวจสบ IDs ไม่ตรงกันแล้ว

# 2. เหตุการณ์: Admin เพิ่มนิยายเข้า List หลัก
top_novels.append("Lord of the Rings")

# 3. ตรวจสอบผลลัพธ์
print(f"Top: {top_novels}")
print(f"Backup: {backup_novels}") # ผลลัพธ์: Backup เปลี่ยนตามเฉยเลย!

Original IDs: 136495098644288 vs 136495098644608
Top & Backup IDs are same?: False
Top: ['Harry Potter', 'Percy Jackson', 'Lord of the Rings']
Backup: ['Harry Potter', 'Percy Jackson']
